In [1]:
import pvlib
import json
import numpy as np
import pandas as pd
import os
import time

# 1. Define some relavent functions

In [5]:
def read_result(CFG):
    # bin_file = CFG['shadow_calc']['shadow_results_path']
    # grid_file = CFG['shadow_calc']['pointgrid_path']
    # gmlid_file = CFG['shadow_calc']['gmlid_path']
    # sunpos_file = CFG['study_area']['solar_positions_path']

    output_root = os.path.join(CFG['study_area']['data_root'], 
                              CFG['shadow_calc']['output_folder_name'])
    bin_file = os.path.join(output_root, 'shadow_result', 'results.bin')
    grid_file = os.path.join(output_root, 'intermediate', 'grid.xyz')
    gmlid_file = os.path.join(output_root, 'shadow_result', 'gmlids.csv')
    sunpos_file = os.path.join(output_root, 'intermediate', 'sun_pos.csv')
    
    with open(bin_file, "rb") as file:
        result_bin = np.fromfile(file, dtype=np.int32)
        num_rows = result_bin[0]
        num_cols = result_bin[1]  
        # Remove the first two elements (number of rows and columns)
        result_bin = result_bin[2:]

        result_bin = result_bin.reshape((num_rows, num_cols))

    point_grid = []
    with open(grid_file, 'r') as file:
        for line in file:
            x, y, z, nx, ny, nz = line.split()
            point_grid.append([float(x), float(y), float(z), float(nx), float(ny), float(nz)])

    point_grid = np.array(point_grid)

    gmlids = pd.read_csv(gmlid_file, header=None)
    gmlids = gmlids[[0]]
    gmlids.columns = ['gmlid']
    sunpos = pd.read_csv(sunpos_file)
    sunpos.columns.values[0]='timestamp'
    gmlid_array = gmlids.to_numpy()

    assert result_bin.shape[0] == point_grid.shape[0]==gmlids.shape[0]
    assert result_bin.shape[1] == sunpos.shape[0]
    print("Num of faces: ", gmlids['gmlid'].nunique())
    print("Num of sample points: ", point_grid.shape[0])
    print("Num of timestamps: ", result_bin.shape[1])
    return result_bin, point_grid, gmlid_array, sunpos
    

In [6]:
def normal2angle(normal_vector):
    # Ensure the normal vector is a NumPy array
    normal_vector = np.array(normal_vector)

    # Calculate the magnitude of the vector
    magnitude = np.linalg.norm(normal_vector)

    # Unpack the normalized vector components
    nx, ny, nz = normal_vector / magnitude


    tilt = np.degrees(np.arccos(nz))


    # Calculate azimuth (angle from north, clockwise in the horizontal plane)
    azimuth = np.degrees(np.arctan2(nx, ny))
    
    # Adjust azimuth to ensure it's between 0 and 360 degrees
    if azimuth < 0:
        azimuth += 360

    return tilt, azimuth

# 2. Main function

## 2.1 obtain relavent data (shadow result)

In [7]:
with open('config.json', 'r') as file:
    CFG = json.load(file)
    

latitude = CFG['study_area']['lat']
longitude = CFG['study_area']['long']
print(latitude)
print(longitude)
result_bin, point_grid, gmlids, sunpos = read_result(CFG) # ndarray, ndarray, df, df

merged_result = np.hstack((result_bin, point_grid, gmlids))

indices = []

# Iterate through gmlids to find start and end indices
start_idx = 0
unique_gmlids = []
unique_gmlids.append(gmlids[0])
for i in range(1, len(gmlids)):
    if gmlids[i] != gmlids[i - 1]:
        unique_gmlids.append(gmlids[i])
        end_idx = i
        indices.append((start_idx, end_idx))
        start_idx = i
# Add the last group
indices.append((start_idx, len(gmlids)))

# Initialize an array to store the aggregated results
aggregated_data = np.empty((len(indices), merged_result.shape[1] - 1))

# Calculate averages for each group
for i, (start, end) in enumerate(indices):
    aggregated_data[i, :] = np.mean(merged_result[start:end, :-1], axis=0)

result_array = np.hstack((aggregated_data[:, :-6], aggregated_data[:, -3:]))



52.285126
6.435649
Num of faces:  42256
Num of sample points:  664112
Num of timestamps:  16


## 2.2 Calculate the hourly irradiation, unit is W/m^2

### 2.2.1 Using tmy data

Obtain and filter the tmy3 file to get the hourly data matched with the shadow result

In [8]:
tmy = pvlib.iotools.get_pvgis_tmy(latitude, longitude)

irradiance_data = tmy[0]
irradiance_data.index = pd.to_datetime(irradiance_data.index)

sunpos['timestamp'] = pd.to_datetime(sunpos['timestamp'], utc=True)

all_ghi = []
all_dni = []
all_dhi = []
all_solar_zenith = []
all_solar_azimuth = []
for index, row in sunpos.iterrows():
        row_time = row['timestamp'].tz_convert(irradiance_data.index.tz)
        month, day, hour = row_time.month, row_time.day, row_time.hour
        match = irradiance_data[(irradiance_data.index.month == month) & 
                    (irradiance_data.index.day == day) & 
                    (irradiance_data.index.hour == hour)]
        solar_zenith = row['apparent_zenith']
        solar_azimuth = row['azimuth']
        if (not match.empty):
            ghi, dni, dhi = match.iloc[0][['ghi', 'dni', 'dhi']]
            all_ghi.append(ghi)
            all_dni.append(dni)
            all_dhi.append(dhi)
            all_solar_zenith.append(solar_zenith)
            all_solar_azimuth.append(solar_azimuth)
        else:
            print("error finding match")

Calculate the hourly irradiation, unit is W/m^2

In [9]:
result_arr = []
start_time = time.time()
for i in range(result_array.shape[0]):
    point_result = []
    surface_normal = result_array[i][-3:]
    surface_tilt, surface_azimuth = normal2angle(surface_normal)
    for j in range(len(all_ghi)):
            poa = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, 
                                                        all_solar_zenith[j],
                                                        all_solar_azimuth[j], 
                                                        all_dni[j], all_ghi[j], all_dhi[j])
            direct = poa['poa_direct']
            diffuse = poa['poa_diffuse']
            irradiance = direct * result_array[i][index] + diffuse
            point_result.append(irradiance)

    result_arr.append(point_result)
    if i%1000==0 and i > 0:
        print("{} surfaces completed".format(i))
        elapsed_time = time.time() - start_time
        average_time_per_iteration = elapsed_time / i
        remaining_iterations = result_array.shape[0] - i
        eta_seconds = remaining_iterations * average_time_per_iteration
        eta_minutes, eta_seconds = divmod(eta_seconds, 60)  # Convert ETA to minutes and seconds
        print("{} surfaces completed. ETA: {:.0f}m {:.0f}s".format(i, eta_minutes, eta_seconds))
        
        
total_arr = np.array(result_arr)

1000 surfaces completed
1000 surfaces completed. ETA: 0m 22s
2000 surfaces completed
2000 surfaces completed. ETA: 0m 22s
3000 surfaces completed
3000 surfaces completed. ETA: 0m 22s
4000 surfaces completed
4000 surfaces completed. ETA: 0m 21s
5000 surfaces completed
5000 surfaces completed. ETA: 0m 21s
6000 surfaces completed
6000 surfaces completed. ETA: 0m 20s
7000 surfaces completed
7000 surfaces completed. ETA: 0m 20s
8000 surfaces completed
8000 surfaces completed. ETA: 0m 20s
9000 surfaces completed
9000 surfaces completed. ETA: 0m 19s
10000 surfaces completed
10000 surfaces completed. ETA: 0m 19s
11000 surfaces completed
11000 surfaces completed. ETA: 0m 18s
12000 surfaces completed
12000 surfaces completed. ETA: 0m 18s
13000 surfaces completed
13000 surfaces completed. ETA: 0m 17s
14000 surfaces completed
14000 surfaces completed. ETA: 0m 16s
15000 surfaces completed
15000 surfaces completed. ETA: 0m 16s
16000 surfaces completed
16000 surfaces completed. ETA: 0m 15s
17000 surf

In [12]:
unique_gmlids_arr = np.array(unique_gmlids)

### 2.2.2 Using pvgis tool

In [ ]:
# result_arr = []
# for i in range(result_array.shape[0]):
#     point_result = []
#     surface_normal = result_array[i][-3:]
#     surface_tilt, surface_azimuth = normal2angle(surface_normal)
#     for j in range(len(all_ghi)):
#             poa = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, 
#                                                         all_solar_zenith[j],
#                                                         all_solar_azimuth[j], 
#                                                         all_dni[j], all_ghi[j], all_dhi[j])
#             direct = poa['poa_global']
#             diffuse = poa['poa_diffuse']
#             irradiance = direct * result_array[i][index] + diffuse
#             point_result.append(irradiance)

#     result_arr.append(point_result)
#     if i%1000==0:
#         print("{} surfaces completed".format(i))
        
        
# total_arr = np.array(result_arr)

In [19]:
# surface_normal = result_array[0][-3:]
# surface_tilt, surface_azimuth = normal2angle(surface_normal)
# import time
# start_time = time.time()
# pv_result = pvlib.iotools.get_pvgis_hourly(latitude, longitude, CFG['study_area']['start_time'],
#                                           CFG['study_area']['end_time'], surface_tilt=surface_tilt, 
#                                            surface_azimuth = surface_azimuth)
# end_time = time.time()
# print(f"Function running time: {end_time - start_time} seconds")

# start_time = time.time()
# poa = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, 
#                                             all_solar_zenith[0],
#                                             all_solar_azimuth[0], 
#                                             all_dni[0], all_ghi[0], all_dhi[0])
# end_time = time.time()
# print(f"Function running time: {end_time - start_time} seconds")

Function running time: 3.1554272174835205 seconds
Function running time: 0.0 seconds


In [13]:
# pv_result

(                           poa_direct  poa_sky_diffuse  poa_ground_diffuse  \
 time                                                                         
 2013-01-01 00:11:00+00:00         0.0              0.0                 0.0   
 2013-01-01 01:11:00+00:00         0.0              0.0                 0.0   
 2013-01-01 02:11:00+00:00         0.0              0.0                 0.0   
 2013-01-01 03:11:00+00:00         0.0              0.0                 0.0   
 2013-01-01 04:11:00+00:00         0.0              0.0                 0.0   
 ...                               ...              ...                 ...   
 2014-12-31 19:11:00+00:00         0.0              0.0                 0.0   
 2014-12-31 20:11:00+00:00         0.0              0.0                 0.0   
 2014-12-31 21:11:00+00:00         0.0              0.0                 0.0   
 2014-12-31 22:11:00+00:00         0.0              0.0                 0.0   
 2014-12-31 23:11:00+00:00         0.0              

# 3. Save result to npz file

In [15]:

save_folder = os.path.join(CFG['study_area']['data_root'], 
                          CFG['shadow_calc']['output_folder_name'], 'irradiance_result')
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder, "hourly_irradiance.npz")
np.savez(save_path, gmlids=unique_gmlids_arr, hourly_irradiance=total_arr)

# 4. Load saved npz file

In [16]:
result_test = np.load(save_path, allow_pickle=True)

In [17]:
gmlid_list = result_test['gmlids']

In [18]:
irradiance_data = result_test['hourly_irradiance']

In [19]:
irradiance_data.shape

(42256, 16)

In [20]:
gmlid_list.shape

(42256, 1)